In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import accuracy_score
import math
import random
from sklearn.manifold import TSNE
from torch.utils.data import Dataset, DataLoader
from sklearn.manifold import TSNE
import cv2

In [2]:
df_raw = pd.read_csv("./seismic.csv", header=None)
df_raw

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0,0.0,0,0.005633,0.006864,0.019389,0.018201,0.0,0.111111,0.0,0.142857,0.0,0,0,0,0,0.004975,0.0050,1
1,0,0.0,0,0.003063,0.006200,0.011186,0.019272,0.0,0.000000,0.0,0.000000,0.0,0,0,0,0,0.000000,0.0000,1
2,0,0.0,0,0.011065,0.037422,0.054437,0.145610,0.0,0.111111,0.0,0.142857,0.0,0,0,0,0,0.007463,0.0075,1
3,0,0.0,0,0.004831,0.012179,0.024609,0.047109,0.0,0.000000,0.0,0.000000,0.0,0,0,0,0,0.000000,0.0000,1
4,0,0.0,1,0.024527,0.042737,0.017151,0.033191,0.0,0.000000,0.0,0.000000,0.0,0,0,0,0,0.000000,0.0000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2362,1,0.0,1,0.031327,0.173384,0.393736,0.264454,0.5,0.000000,0.0,0.000000,0.0,0,0,0,0,0.000000,0.0000,1
2363,1,0.0,1,0.016185,0.122453,0.230425,0.229122,0.0,0.000000,0.0,0.000000,0.0,0,0,0,0,0.000000,0.0000,1
2364,1,0.0,1,0.010348,0.119132,0.146905,0.222698,0.0,0.000000,0.0,0.000000,0.0,0,0,0,0,0.000000,0.0000,1
2365,0,0.0,1,0.006176,0.070859,0.073080,0.104925,0.0,0.000000,0.0,0.000000,0.0,0,0,0,0,0.000000,0.0000,1


In [3]:
y = pd.get_dummies(df_raw.iloc[:,-1:].astype(str))
y

,18_-1,18_1
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
2362,0,1
2363,0,1
2364,0,1
2365,0,1


In [4]:
x = df_raw.iloc[:,:-1]
x

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0,0.0,0,0.005633,0.006864,0.019389,0.018201,0.0,0.111111,0.0,0.142857,0.0,0,0,0,0,0.004975,0.0050
1,0,0.0,0,0.003063,0.006200,0.011186,0.019272,0.0,0.000000,0.0,0.000000,0.0,0,0,0,0,0.000000,0.0000
2,0,0.0,0,0.011065,0.037422,0.054437,0.145610,0.0,0.111111,0.0,0.142857,0.0,0,0,0,0,0.007463,0.0075
3,0,0.0,0,0.004831,0.012179,0.024609,0.047109,0.0,0.000000,0.0,0.000000,0.0,0,0,0,0,0.000000,0.0000
4,0,0.0,1,0.024527,0.042737,0.017151,0.033191,0.0,0.000000,0.0,0.000000,0.0,0,0,0,0,0.000000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2362,1,0.0,1,0.031327,0.173384,0.393736,0.264454,0.5,0.000000,0.0,0.000000,0.0,0,0,0,0,0.000000,0.0000
2363,1,0.0,1,0.016185,0.122453,0.230425,0.229122,0.0,0.000000,0.0,0.000000,0.0,0,0,0,0,0.000000,0.0000
2364,1,0.0,1,0.010348,0.119132,0.146905,0.222698,0.0,0.000000,0.0,0.000000,0.0,0,0,0,0,0.000000,0.0000
2365,0,0.0,1,0.006176,0.070859,0.073080,0.104925,0.0,0.000000,0.0,0.000000,0.0,0,0,0,0,0.000000,0.0000


In [5]:
x_raw, x_test, y_raw, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

p_count = y_raw['18_1'][y_raw['18_1'] == 1].count()
n_count = y_raw['18_-1'][y_raw['18_-1'] == 1].count()

print("Positive: ", p_count)
print("Negative: ",n_count)

new_data = p_count - n_count
print("Negative Data needed", new_data)

Positive:  1791
Negative:  102
Negative Data needed 1689


In [6]:
class VariationalAutoencoder(nn.Module):
    def __init__(self, num_features=10, num_dim=18):
        super(VariationalAutoencoder, self).__init__()
        
        self.num_features = num_features
        self.num_dim = num_dim
        
        self.encoder_layer_1 = nn.Linear(in_features=self.num_dim, out_features=10)
        self.encoder_layer_2 = nn.Linear(in_features=10, out_features=(self.num_features * 2))
        
        self.decoder_layer_1 = nn.Linear(in_features=self.num_features, out_features=10)
        self.decoder_layer_2 = nn.Linear(in_features=10, out_features=self.num_dim)
        
        self.relu = nn.ReLU() # hidden layers
        self.sigmoid = nn.Sigmoid() # output layer
        
    def reparameterize(self, mu, log_var):
        """
        :param mu: mean from the encoder's latent space
        :param log_var: log variance from the encoder's latent space
        """
        std = torch.exp(0.5*log_var) # standard deviation
        eps = torch.randn_like(std)  # `randn_like` as we need the same size
        sample = mu + (eps * std)    # sampling as if coming from the input space
        
        return sample
    
    def encode(self, x):
        # encoding
        #x = F.relu(self.encoder_layer_1(x))
        x = self.encoder_layer_1(x)
        x = self.relu(x)
        x = self.encoder_layer_2(x).view(-1, 2, self.num_features)
        
        # get `mu` and `log_var`
        mu = x[:, 0, :] # the first feature values as mean
        log_var = x[:, 1, :] # the other feature values as variance
        
        # get the latent vector through reparameterization
        z = self.reparameterize(mu, log_var)
        
        return z, mu, log_var
    
    def decode(self, z, mu, log_var):
        # decoding
        #x = F.relu(self.decoder_layer_1(z))
        x = self.decoder_layer_1(z)
        x = self.relu(x)
        
        #reconstruction = torch.sigmoid(self.decoder_layer_2(x))
        reconstruction = self.decoder_layer_2(x)
        reconstruction = self.sigmoid(reconstruction)
        
        return reconstruction, mu, log_var
    
    # Utility function to generate new data based on:
    # mu: The average that you want to have (should be the same size as num_features)
    # log_var: The variance that you want to have (should be the same size as num_features)
    def sample(self, mu, log_var):
        z = self.reparameterize(mu, log_var)
        reconstruction, mu, log_var = self.decode(z, mu, log_var)
        
        return reconstruction
        
    
    def forward(self, x):
        
        z, mu, log_var = self.encode(x)
        reconstruction, mu, log_var = self.decode(z, mu, log_var)
        
        return reconstruction, mu, log_var

In [7]:
class AutoencoderDataset(Dataset):
    def __init__(self, x):
        self.x = x
        
    def __len__(self):
        return len(self.x)
    
    # Requires you to return data as a pair of _x, _y
    def __getitem__(self, index):
        return self.x[index], self.x[index]

In [8]:
def final_loss(bce_loss, mu, logvar):
    BCE = bce_loss 
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [9]:
num_features = 10
model = VariationalAutoencoder(num_features=num_features)

In [10]:
learning_rate = 0.001
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
batch_size = 3

In [11]:
# Define the training process
# Each call of the train function == 1 Epoch
def train_fn(loader, model, optimizer, loss_fn, batch_size):
    loop = tqdm(loader)
    
    count = 0
    ave_loss = 0.00
    
    # Loop per batch
    for batch_idx, (data, targets) in enumerate(loop):
        reconstruction, mu, logvar = model.forward(data)
        
        loss = loss_fn(reconstruction, targets)
        
        loss = final_loss(loss, mu, logvar)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
        loop.set_postfix(loss=loss.item())
        
        ave_loss += loss.item()
        count += 1
        
    ave_loss = ave_loss / count
    
    return ave_loss

In [12]:
x_missing = x_raw[y_raw['18_-1'] == 1]
x_missing = torch.Tensor(np.array(x_missing))

# use Dataloader for Autoencoder 
custom_dataset = AutoencoderDataset(x_missing)

train_loader = DataLoader(
    custom_dataset,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False
)

In [13]:
epochs = 100
losses = []

for epoch in range(epochs):
    print("Epoch: {}".format(epoch))
    
    ave_loss = train_fn(
        train_loader,
        model,
        optimizer,
        loss_fn,
        batch_size
    )
    
    losses.append(ave_loss)
    
    print("Ave Loss: {}".format(ave_loss))
    
    state = { 'state_dict': model.state_dict() }

    torch.save(state, "variational-autoencoder.pth")

Epoch: 0


100%|██████████| 34/34 [00:00<00:00, 328.84it/s, loss=1.1]


Ave Loss: 1.2656840927460615
Epoch: 1


100%|██████████| 34/34 [00:00<00:00, 360.65it/s, loss=0.879]


Ave Loss: 0.9244601849247428
Epoch: 2


...

Ave Loss: 0.2602260840289733
Epoch: 98


100%|██████████| 34/34 [00:00<00:00, 442.82it/s, loss=0.265]


Ave Loss: 0.2599586787469247
Epoch: 99


100%|██████████| 34/34 [00:00<00:00, 305.24it/s, loss=0.268]

Ave Loss: 0.25975781810634274


In [14]:
# Create a vector of zero tensors representing 0 average per num_feature (right in the middle of the dist)
sampled_mu = torch.Tensor([np.zeros(num_features)])

# Create a vector of zero tensors representing 0 standard deviations away from the mean to create variations
# Change this is you want to sample away from the mean to create "off-quality" data
sampled_logvar = torch.Tensor([np.zeros(num_features)])

# print(sampled_logvar)

sampled_logvar.shape

C:\Users\JBASCO\AppData\Local\Temp\ipykernel_9352\602494829.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:233.)
  sampled_mu = torch.Tensor([np.zeros(num_features)])


torch.Size([1, 10])

In [15]:
# Grab a reconstructed sample from the trained model
# NOTE: reconstruction returns as an array because it's possible to have multiple predictions later on
reconstruction = model.sample(sampled_mu, sampled_logvar)

In [16]:
added_m_data = []

for i in range(new_data):
    reconstruction = model.sample(sampled_mu, sampled_logvar)
    reconstruction = added_m_data.append(reconstruction[0].detach().cpu().numpy())

col_name = [x for x in range(0, 18)]
x_added = pd.DataFrame(added_m_data, columns=col_name)
x_added

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.490822,0.236704,0.838328,0.022572,0.129962,0.098845,0.114645,0.053651,0.207368,0.140402,0.126517,0.040874,0.007934,0.002075,0.001391,0.001484,0.024966,0.020336
1,0.502023,0.265323,0.851260,0.031490,0.148548,0.097934,0.125338,0.061071,0.221899,0.122147,0.140092,0.043835,0.008338,0.002044,0.001005,0.001876,0.019309,0.017996
2,0.485260,0.264985,0.879468,0.026871,0.145319,0.084251,0.122507,0.067632,0.230851,0.096657,0.152628,0.046282,0.010472,0.001792,0.000837,0.001446,0.017537,0.012965
3,0.490971,0.205532,0.902000,0.011356,0.103924,0.062386,0.089955,0.037738,0.175078,0.098923,0.105699,0.020016,0.005803,0.000347,0.000288,0.000199,0.007332,0.004836
4,0.459627,0.261790,0.885970,0.011495,0.104771,0.075157,0.096171,0.028629,0.192594,0.094801,0.114748,0.022342,0.002642,0.001034,0.000256,0.000505,0.009012,0.004611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1684,0.529182,0.260423,0.844457,0.020530,0.127021,0.100457,0.124481,0.061057,0.185129,0.130270,0.115371,0.045376,0.005703,0.002405,0.000829,0.001279,0.018968,0.019901
1685,0.475950,0.238174,0.848658,0.024260,0.144674,0.097085,0.125367,0.053414,0.214993,0.130233,0.139550,0.034710,0.010369,0.002966,0.000780,0.001577,0.021386,0.014021
1686,0.490222,0.254216,0.842856,0.020945,0.138176,0.089499,0.126790,0.050854,0.218791,0.121472,0.131746,0.040593,0.007555,0.002839,0.000905,0.001594,0.022694,0.015315
1687,0.472857,0.229411,0.899982,0.022851,0.134631,0.071085,0.098017,0.032179,0.222900,0.094840,0.106217,0.022745,0.005672,0.000645,0.000249,0.000657,0.010318,0.006446


In [17]:
col_name = ['18_-1', '18_1']

y_targets = [(0,1) for x in range(new_data)]
y_added = pd.DataFrame(y_targets, columns=col_name)

y_added

,18_-1,18_1
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
1684,0,1
1685,0,1
1686,0,1
1687,0,1


In [18]:
x_fab = x_raw

x_fab = pd.concat([x_fab, x_added])
y_fab = pd.concat([y_raw, y_added])

x_fab =  torch.Tensor(x_fab.values)
x_test =  torch.Tensor(x_test.values)
y_fab  = torch.Tensor(y_fab.values)


In [19]:
class NeuralNetwork(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        
        self.hidden_layer_1 = nn.Linear(self.in_dim, 12) # input to first hidden layer
        self.hidden_layer_2 = nn.Linear(12, 2)
        self.output_layer = nn.Linear(2, self.out_dim)
        self.activation = nn.Sigmoid()
    
    def forward(self, x):
        x = self.hidden_layer_1(x)
        x = self.activation(x)
        
        x = self.hidden_layer_2(x)
        x = self.activation(x)
        
        y = self.output_layer(x)
        y = self.activation(y)
        
        return y

In [20]:
model = NeuralNetwork(18, 2)

predictions = model.forward(x_fab)

predictions.shape

torch.Size([3582, 2])

In [21]:
learning_rate = 0.001
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
batch_size = 5

In [22]:
# Custom dataset to treat how the model picks an x, y combination from the dataset
class MyCustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __len__(self):
        return len(self.x)
    
    # Requires you to return data as a pair of _x, _y
    def __getitem__(self, index):
        return self.x[index], self.y[index]

In [23]:
# Define the training process
# Each call of the train function == 1 Epoch
def train_nn(loader, model, optimizer, loss_fn, batch_size):
    loop = tqdm(loader)
    
    count = 0
    ave_loss = 0.00
    
    # Loop per batch
    for batch_idx, (data, targets) in enumerate(loop):
        predictions = model.forward(data)
        
        loss = loss_fn(predictions, targets)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
        loop.set_postfix(loss=loss.item())
        
        ave_loss += loss.item()
        count += 1
        
    ave_loss = ave_loss / count
    
    return ave_loss


In [24]:
custom_dataset = MyCustomDataset(x=x_fab, y=y_fab)

train_loader = DataLoader(
    custom_dataset,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False
)

In [25]:
epochs = 100
losses = []

for epoch in range(epochs):
    print("Epoch: {}".format(epoch))
    
    ave_loss = train_nn(
        train_loader,
        model,
        optimizer,
        loss_fn,
        batch_size
    )
    
    losses.append(ave_loss)
    
    print("Ave Loss: {}".format(ave_loss))
    
    state = { 'state_dict': model.state_dict() }

    torch.save(state, "seismic_model2.pth")

Epoch: 0


100%|██████████| 717/717 [00:01<00:00, 565.96it/s, loss=0.015] 


Ave Loss: 0.07438789996756254
Epoch: 1


100%|██████████| 717/717 [00:01<00:00, 556.80it/s, loss=0.00551]


Ave Loss: 0.0322035767990093
Epoch: 2


...

Ave Loss: 0.026033280588252303
Epoch: 98


100%|██████████| 717/717 [00:01<00:00, 514.30it/s, loss=0.00052] 


Ave Loss: 0.02602359548821101
Epoch: 99


100%|██████████| 717/717 [00:01<00:00, 428.58it/s, loss=0.00052] 

Ave Loss: 0.026014106319063247


In [26]:
# Predictions 
model = NeuralNetwork(18, 2)

state = torch.load("seismic_model2.pth")

model.load_state_dict(state['state_dict'])

predictions = model.forward(x_test)

predictions = predictions.detach().cpu().numpy()

predictions = [[1, 0] if y[0] > y[1] else [0, 1] for y in predictions]

In [27]:
# Accuracy score
print(accuracy_score(y_test, predictions))

0.9514767932489452


In [28]:
# Classification Report
print(classification_report(predictions, y_test))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.95      0.98       474

   micro avg       0.95      0.95      0.95       474
   macro avg       0.50      0.48      0.49       474
weighted avg       1.00      0.95      0.98       474
 samples avg       0.95      0.95      0.95       474



c:\Training\Machine Learning\day4\env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
